In [1]:
from google.colab import userdata
my_secret_key = userdata.get('hf-llama')


In [2]:
!pip install -q -U "bitsandbytes" "transformers>=4.40.0" "datasets" "peft" "accelerate" "huggingface_hub"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from huggingface_hub import login
login()  # حط هنا hf_xxx اللي من حسابك


In [4]:
from google.colab import files
uploaded = files.upload()  # اختار concatenated_datasets.jsonl أو أي اسم عندك
print(uploaded.keys())


Saving preprocessed_datasets.jsonl to preprocessed_datasets.jsonl
dict_keys(['preprocessed_datasets.jsonl'])


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=my_secret_key)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [6]:
from datasets import load_dataset

DATA_PATH = "/content/preprocessed_datasets.jsonl"
ds = load_dataset("json", data_files={"train": DATA_PATH}, split="train")
print(ds[0])


Generating train split: 0 examples [00:00, ? examples/s]

{'messages': [{'role': 'user', 'content': 'Say, Jim, how about going for a few beers after dinner?'}, {'role': 'assistant', 'content': 'You know that is tempting but is really not good for our fitness.'}, {'role': 'user', 'content': 'What do you mean? It will help us to relax.'}, {'role': 'assistant', 'content': "Do you really think so? I don't. It will just make us fat and act silly. Remember last time?"}, {'role': 'user', 'content': "I guess you are right. But what shall we do? I don't feel like sitting at home."}, {'role': 'assistant', 'content': 'I suggest a walk over to the gym where we can play singsong and meet some of our friends.'}, {'role': 'user', 'content': "That's a good idea. I hear Mary and Sally often go there to play pingpong. Perhaps we can make a foursome with them."}, {'role': 'assistant', 'content': 'Sounds great to me! If they are willing, we could ask them to go dancing with us. That is excellent exercise and fun, too.'}, {'role': 'user', 'content': "Good. Let's 

In [7]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model

# 1) إعدادات أساسية
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
DATA_PATH = "/content/preprocessed_datasets.jsonl"

# 2) تحميل الداتا وتقسيمها
dataset = load_dataset(
    "json",
    data_files={"train": DATA_PATH},
    split="train",
)

dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_ds = dataset["train"]
val_ds = dataset["test"]

print("Example row:", train_ds[0])

# 3) تحميل التوكنيزر والموديل بــ 4bit (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=my_secret_key)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    # device_map="auto", # Removed device_map to let Trainer handle device placement
    token=my_secret_key
)

# 4) إعداد LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 5) تجهيز النص من messages باستخدام chat_template بتاع LLaMA

def format_example(example):
    messages = example["messages"]  # زي الـ Example اللي انت مطبعه

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,  # بنعلّم الموديل على الحوار بالكامل
    )
    return {"text": text}

train_ds = train_ds.map(format_example)
val_ds = val_ds.map(format_example)

print("Formatted text sample:\n", train_ds[0]["text"][:400])

# 6) التوكنيز + labels

def tokenize(example):
    enc = tokenizer(
        example["text"],
        truncation=True,
        max_length=512,   # زوّد لـ 1024 لو GPU مستحمل
        padding="max_length",
    )
    enc["labels"] = enc["input_ids"].copy()
    return enc

train_tokenized = train_ds.map(
    tokenize,
    batched=True,
    remove_columns=train_ds.column_names,
)
val_tokenized = val_ds.map(
    tokenize,
    batched=True,
    remove_columns=val_ds.column_names,
)

# 7) إعدادات التدريب
training_args = TrainingArguments(
    output_dir="/content/llama3-lora-ft",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=200,
    logging_steps=20,
    learning_rate=2e-4,
    bf16=True, # Changed from fp16=True to bf16=True for TPU compatibility
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.01,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)

trainer.train()

SAVE_DIR = "/content/llama3-lora-ft-final"
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print(f"Saved fine-tuned model to {SAVE_DIR}")

Example row: {'messages': [{'role': 'user', 'content': 'Good morning, sir. Could I be of any service to you?'}, {'role': 'assistant', 'content': 'Have you got any Chinese paintings?'}, {'role': 'user', 'content': 'Yes, we have landscape paintings, figure paintings and flower-and-bird paintings.'}, {'role': 'assistant', 'content': "I'd like a flower-and-bird painting."}, {'role': 'user', 'content': 'Good. How about this picture of cranes with pine trees? It was painted by a famous Chinese painter in King Dynasty.'}, {'role': 'assistant', 'content': 'Oh, they look just like live cranes. I like this picture very much. How much is it?'}, {'role': 'user', 'content': 'One thousand dollars.'}, {'role': 'assistant', 'content': "It's a lot of money. But the picture is excellent, I'll take it."}]}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


Map:   0%|          | 0/12484 [00:00<?, ? examples/s]

Map:   0%|          | 0/1388 [00:00<?, ? examples/s]

Formatted text sample:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Good morning, sir. Could I be of any service to you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Have you got any Chinese paintings?<|eot_id|><|start_header_id|>user<|end_header_id|>

Yes, we have landscape paint


Map:   0%|          | 0/12484 [00:00<?, ? examples/s]

Map:   0%|          | 0/1388 [00:00<?, ? examples/s]